自然言語処理以外の特徴量作成用notebook

# Libraries

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import random
import pandas as pd
import os
import time
import numpy as np
from sklearn.model_selection import StratifiedKFold
import itertools
from sklearn.metrics import mean_squared_error
import typing as tp
from pathlib import Path
from contextlib import contextmanager
from sklearn.model_selection import KFold
import time
from matplotlib import pyplot as plt
import seaborn as sns
import datetime
import gc
import re
from pandas.core.common import flatten
import collections
from collections import Counter
import gensim
from bs4 import BeautifulSoup
from gensim.models import word2vec, KeyedVectors
from tqdm import tqdm

tqdm.pandas()

# Utils

In [ ]:
@contextmanager
def timer(name: str):
    t0 = time.time()
    print(f"[{name}] start")
    yield
    msg = f"[{name}] done in {time.time() - t0:.0f} s"
    print(msg)
    
    
def set_seed(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    
    
set_seed(42)

# Data Loading

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd "drive/My Drive/probspace_citations"

Mounted at /content/drive
/content/drive/My Drive/probspace_citations


In [ ]:
DATADIR = Path('input')
OUTDIR = Path('features')

データは、citesのあるtrainとtestにcites_known = 1, その他にはcites_known = 0としてpickle化してあるものをダウンロード

In [ ]:
with timer("Load data"):
    ALL_df = pd.read_pickle(DATADIR / 'ALL_df.pkl') # 全てのtrain/testを結合したデータ

[Load data] start
[Load data] done in 26 s


In [ ]:
ALL_df.head(3)

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed,doi_cites,cites,cites_known
0,hep-ph/9902295,Michael Kraemer,Mark E. Hayes (University College London) and ...,Heavy-Flavour Production at HERA,"LaTeX, 21 pages, 13 Postscript figures. Summar...","J.Phys.G25:1477-1493,1999",10.1088/0954-3899/25/7/332,"CERN-TH/99-30, UCL/HEP 99-03",hep-ph hep-ex,None,We review the theoretical and experimental s...,"[{'version': 'v1', 'created': 'Wed, 10 Feb 199...",2008-11-26,"[[Hayes, Mark E., , University College London]...",1,NaN,0
1,1403.7138,Aigen Li,"Qi Li, S.L. Liang, Aigen Li (University of Mis...",Spectropolarimetric Constraints on the Nature ...,"5 pages, 2 figures; accepted for publication i...",None,10.1093/mnrasl/slu021,None,astro-ph.GA,http://arxiv.org/licenses/nonexclusive-distrib...,While it is well recognized that interstella...,"[{'version': 'v1', 'created': 'Thu, 27 Mar 201...",2015-06-19,"[[Li, Qi, , University of Missouri], [Liang, S...",8,7.0,1
2,1405.5857,Michael Mortonson,"Michael J. Mortonson, Uro\v{s} Seljak",A joint analysis of Planck and BICEP2 B modes ...,"13 pages, 4 figures; submitted to JCAP; refere...",JCAP10(2014)035,10.1088/1475-7516/2014/10/035,None,astro-ph.CO gr-qc hep-ph hep-th,http://arxiv.org/licenses/nonexclusive-distrib...,We analyze BICEP2 and Planck data using a mo...,"[{'version': 'v1', 'created': 'Thu, 22 May 201...",2014-10-17,"[[Mortonson, Michael J., ], [Seljak, Uroš, ]]",122,188.0,1


In [ ]:
with timer("Load data"):
    df = pd.read_pickle(DATADIR / 'df.pkl') # citesの値が与えられたtrainとtestを結合したデータ

[Load data] start
[Load data] done in 4 s


In [ ]:
df.head(3)

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed,doi_cites,cites,cites_known
0,1403.7138,Aigen Li,"Qi Li, S.L. Liang, Aigen Li (University of Mis...",Spectropolarimetric Constraints on the Nature ...,"5 pages, 2 figures; accepted for publication i...",None,10.1093/mnrasl/slu021,None,astro-ph.GA,http://arxiv.org/licenses/nonexclusive-distrib...,While it is well recognized that interstella...,"[{'version': 'v1', 'created': 'Thu, 27 Mar 201...",2015-06-19,"[[Li, Qi, , University of Missouri], [Liang, S...",8,7.0,1
1,1405.5857,Michael Mortonson,"Michael J. Mortonson, Uro\v{s} Seljak",A joint analysis of Planck and BICEP2 B modes ...,"13 pages, 4 figures; submitted to JCAP; refere...",JCAP10(2014)035,10.1088/1475-7516/2014/10/035,None,astro-ph.CO gr-qc hep-ph hep-th,http://arxiv.org/licenses/nonexclusive-distrib...,We analyze BICEP2 and Planck data using a mo...,"[{'version': 'v1', 'created': 'Thu, 22 May 201...",2014-10-17,"[[Mortonson, Michael J., ], [Seljak, Uroš, ]]",122,188.0,1
2,1807.01034,Evangelos Thomas Karamatskos,"Evangelos T. Karamatskos, Sebastian Raabe, Ter...",Molecular movie of ultrafast coherent rotation...,9 Figures,"Nat Commun 10, 3364 (2019)",10.1038/s41467-019-11122-y,None,physics.chem-ph physics.atom-ph quant-ph,http://arxiv.org/licenses/nonexclusive-distrib...,Recording molecular movies on ultrafast time...,"[{'version': 'v1', 'created': 'Tue, 3 Jul 2018...",2020-05-19,"[[Karamatskos, Evangelos T., ], [Raabe, Sebast...",6,8.0,1


# Basic features

## Pseudo category

In [ ]:
def make_id_feat(x):
    try: return re.search('(.*)/(.*)', x).group(1)
    except: return x[:4]
def make_doi_feat(x):
    try: return float(x.split('/')[0])
    except: return np.nan
def make_doi_feat_2(x):
    try: return float(x.split('/')[1])
    except: return np.nan

with timer("pseudo category"):
    ALL_df['id_feat'] = ALL_df['id'].apply(make_id_feat)
    ALL_df['doi_feat'] = ALL_df['doi'].apply(make_doi_feat)
    ALL_df['doi_feat_2'] = ALL_df['doi'].apply(make_doi_feat_2)
    df['id_feat'] = df['id'].apply(make_id_feat)
    df['doi_feat'] = df['doi'].apply(make_doi_feat)
    df['doi_feat_2'] = df['doi'].apply(make_doi_feat_2)

[pseudo category] start
[pseudo category] done in 5 s


## Time features

In [ ]:
def time_preprocess(input_df):
    output_df = input_df.copy()

    output_df["first_created_unixtime"] = pd.to_datetime(input_df.versions.apply(lambda p: p[0]["created"])).astype(int) / 1e9
    output_df["last_created_unixtime"] = pd.to_datetime(input_df.versions.apply(lambda p: p[-1]["created"])).astype(int) / 1e9
    output_df["diff_created_unixtime"] = output_df["last_created_unixtime"] - output_df["first_created_unixtime"]
    output_df["num_created"] = input_df.versions.apply(lambda p: len(p))
    output_df["from_first_created"] = (pd.Timestamp(year=2021,month=1,day=1,hour=0,minute=0,second=0,tz='UTC') - pd.to_datetime(input_df.versions.apply(lambda p: p[0]["created"]))).astype(int) / 1e9
    output_df["from_last_created"] = (pd.Timestamp(year=2021,month=1,day=1,hour=0,minute=0,second=0,tz='UTC') - pd.to_datetime(input_df.versions.apply(lambda p: p[-1]["created"]))).astype(int) / 1e9
    output_df["from_update"] = (pd.Timestamp(year=2021,month=1,day=1) - pd.to_datetime(df.update_date)).astype(int) / 1e9
    return output_df

with timer("time features"):
    ALL_df = time_preprocess(ALL_df)
    df = time_preprocess(df)

[time features] start
[time features] done in 673 s


## Length features

In [ ]:
with timer('letter number'):
    df['title_letter_number'] = df['title'].map(lambda x: len(x))
    df['abstract_letter_number'] = df['abstract'].map(lambda x: len(x))
    
with timer('word number'):
    df['title_word_number'] = df['title'].map(lambda x: len(x.split(' ')))
    df['abstract_letter_number'] = df['abstract'].map(lambda x: len(x.split(' ')))
    df['categories_number'] = df['categories'].map(lambda x: len(x.split(' ')))

[letter number] start
[letter number] done in 0 s
[word number] start
[word number] done in 1 s


## Regex features

In [ ]:
with timer('regex features'):
    df['comments_pages'] = df['comments'].fillna(' ').progress_apply(lambda x: re.findall(r'(\d+)[^\d,]*pages', x)[0] if re.search(r'(\d+)[^\d,]*pages', x) else 0)
    df['comments_figures'] = df['comments'].fillna(' ').progress_apply(lambda x: re.findall(r'(\d+)[^\d,]*figures', x)[0] if re.search(r'(\d+)[^\d,]*figures', x) else 0)

 21%|██        | 15551/74201 [00:00<00:00, 155506.44it/s]

[regex features] start


100%|██████████| 74201/74201 [00:00<00:00, 159460.23it/s]

[regex features] done in 1 s


In [ ]:
df.to_csv(OUTDIR / 'base_df.csv', index=False)

# Multi-word categories features

In [ ]:
df['categories_space_list'] = df['categories'].apply(lambda x: list(x.split()))
df['categories_comma_list'] = df['categories'].apply(lambda x: list(re.split('[. ]', x)))
df.authors_parsed = df.authors_parsed.apply(lambda x: [[y[0]+' '+y[1], y[-1]] for y in x])
df['authors_list'] = df.authors_parsed.apply(lambda x: [i for i in list(itertools.chain.from_iterable(x)) if i!=''])

In [ ]:
ALL_df['categories_space_list'] = ALL_df['categories'].apply(lambda x: list(x.split()))
ALL_df['categories_comma_list'] = ALL_df['categories'].apply(lambda x: list(re.split('[. ]', x)))
ALL_df.authors_parsed = ALL_df.authors_parsed.apply(lambda x: [[y[0]+' '+y[1], y[-1]] for y in x])
ALL_df['authors_list'] = ALL_df.authors_parsed.apply(lambda x: [i for i in list(itertools.chain.from_iterable(x)) if i!=''])

In [ ]:
def multiple_word_categories_features(input_df, target_cols):
    output_df = input_df[['id']]

    for target_col in target_cols:

        if input_df.cites_known.min() == 1:
            new_cols = [f'Multiple_{target_col}_count', f'Multiple_{target_col}_cumave', f'Multiple_{target_col}_anticipator']
        else:
            new_cols = [f'ALL_Multiple_{target_col}_count', f'ALL_Multiple_{target_col}_cumave', f'Multiple_{target_col}_anticipator']

        duplicates_containing_list = list(itertools.chain.from_iterable(list(input_df[target_col])))
        count_dic = collections.Counter(duplicates_containing_list)

        def categories_count_sum(l):
            sum = 0
            for i in l: 
                sum += count_dic[i]
            return sum

        keys = list(set(duplicates_containing_list))
        values = [0]*len(keys)
        doi_cites_cumsum_dic = dict(zip(keys, values))
        doi_cites_cumcount_dic = dict(zip(keys, values))

        keys = list(input_df.id)
        values = [0]*len(keys)
        id_cumave_dic = dict(zip(keys, values))
        id_anticipator_dic = dict(zip(keys, values))


        for target_list, doi_cites, id in tqdm(zip(input_df.sort_values('first_created_unixtime')[target_col], input_df.sort_values('first_created_unixtime')['doi_cites'], input_df.sort_values('first_created_unixtime')['id'])):
            cumave = 0
            anticipator = 0

            for i in target_list:
                doi_cites_cumsum_dic[i] += doi_cites
                doi_cites_cumcount_dic[i] += 1
                cumave += doi_cites_cumsum_dic[i]/doi_cites_cumcount_dic[i]
                anticipator += count_dic[i]/doi_cites_cumcount_dic[i]
            
            id_cumave_dic[id] = cumave
            id_anticipator_dic[id] = anticipator

        new_df = pd.DataFrame()
        new_df[new_cols[0]] = input_df[target_col].apply(categories_count_sum)
        new_df[new_cols[1]] = input_df['id'].map(id_cumave_dic)
        new_df[new_cols[2]] = input_df['id'].map(id_anticipator_dic)
        output_df = pd.concat([output_df, new_df], axis=1)

    return output_df

In [ ]:
with timer('Multiple word categories features'):
    ALL_multiple_word_categories_features = multiple_word_categories_features(ALL_df, ['categories_space_list', 'categories_comma_list', 'authors_list'])
    multiple_word_categories_features = multiple_word_categories_features(df, ['categories_space_list', 'categories_comma_list', 'authors_list'])

[Multiple word categories features] start


910608it [00:03, 244025.76it/s]
910608it [00:04, 208855.38it/s]
910608it [00:12, 75199.87it/s]
74201it [00:00, 285475.64it/s]
74201it [00:00, 242492.60it/s]
74201it [00:00, 92140.98it/s]


[Multiple word categories features] done in 66 s


In [ ]:
multiple_word_categories_features = pd.merge(multiple_word_categories_features, ALL_multiple_word_categories_features, on='id', how='left')

In [ ]:
multiple_word_categories_features.to_csv(OUTDIR / 'multiple_word_categories_features.csv', index=False)

# Groupby features

In [ ]:
GROUPBY_COLS = ['submitter', 'categories', 'license', 'id_feat', 'doi_feat', 'doi_feat_2']

In [ ]:
def aggregation(input_df, group_keys, group_value, agg_methods):
    output_df = input_df[['id']]
    for agg_method in agg_methods:
        for col in group_keys:
            if callable(agg_method):
                agg_method_name = agg_method.__name__
            else:
                agg_method_name = agg_method
            if input_df.cites_known.min() == 1:
                new_col = f"{agg_method_name}_{group_value}_grpby_{col}"
            else:
                new_col = f"ALL_{agg_method_name}_{group_value}_grpby_{col}"
            new_df = (input_df[[col] + [group_value]].groupby(col)[[group_value]].transform(agg_method))
            new_df.columns = [new_col]
            output_df = pd.concat([output_df, new_df], axis=1)

    return output_df

def aggregation_CumFeat(input_df, group_keys, group_value, ascendings):
    output_df = input_df[['id']]
    for col in group_keys:
        for ascending in ascendings:
            if input_df.cites_known.min() == 1:
                new_col = f"CumFeat_{group_value}_grpby_{col}_{ascending}"
            else:
                new_col = f"ALL_CumFeat_{group_value}_grpby_{col}_{ascending}"
            input_df['lag'] = input_df.sort_values('first_created_unixtime', ascending=ascending).groupby([col])[[group_value]].shift(1)
            cum = input_df[[col] + ['lag', 'first_created_unixtime']].sort_values('first_created_unixtime').groupby(col).lag.agg(['cumsum', 'cumcount'])
            new_df = pd.DataFrame(cum['cumsum'] / cum['cumcount'])
            new_df.columns = [new_col]
            output_df = pd.concat([output_df, new_df], axis=1)

    return output_df

def aggregation_Multiple_CumFeat(input_df, group_keys, group_value, ascendings):
    output_df = input_df[['id']]
    for (col1,col2) in list(itertools.combinations(group_keys,2)):
        for ascending in ascendings:
            if input_df.cites_known.min() == 1:
                new_col = f"CumFeat_{group_value}_grpby_{col1}_{col2}_{ascending}"
            else:
                new_col = f"ALL_CumFeat_{group_value}_grpby_{col1}_{col2}_{ascending}"
            input_df['lag'] = input_df.sort_values('first_created_unixtime', ascending=ascending).groupby([col1,col2])[[group_value]].shift(1)
            cum = input_df[[col1,col2] + ['lag', 'first_created_unixtime']].sort_values('first_created_unixtime').groupby([col1,col2]).lag.agg(['cumsum', 'cumcount'])
            new_df = pd.DataFrame(cum['cumsum'] / cum['cumcount'])
            new_df.columns = [new_col]
            output_df = pd.concat([output_df, new_df], axis=1)

    return output_df

def aggregation_FirstTimeWeighted(input_df, group_keys, group_value, ascendings):
    output_df = input_df[['id']]
    for col in group_keys:
        for ascending in ascendings:
            if input_df.cites_known.min() == 1:
                new_col = f"FirstTimeWeighted_{group_value}_grpby_{col}_{ascending}"
            else:
                new_col = f"ALL_FirstTimeWeighted_{group_value}_grpby_{col}_{ascending}"
            input_df['lag'] = input_df.sort_values('first_created_unixtime', ascending=ascending).groupby(col)[[group_value]].shift(1)
            cum1 = input_df[[col] + ['lag', 'first_created_unixtime']].sort_values('first_created_unixtime').groupby(col).lag.agg(['cumsum'])
            cum2 = input_df[[col] + ['from_first_created', 'first_created_unixtime']].sort_values('first_created_unixtime').groupby(col).from_first_created.agg(['cumsum'])
            new_df = pd.DataFrame(cum1['cumsum'] / cum2['cumsum'])
            new_df.columns = [new_col]
            output_df = pd.concat([output_df, new_df], axis=1)

    return output_df

def aggregation_UpdateTimeWeighted(input_df, group_keys, group_value, ascendings):
    output_df = input_df[['id']]
    for col in group_keys:
        for ascending in ascendings:
            if input_df.cites_known.min() == 1:
                new_col = f"UpdateTimeWeighted_{group_value}_grpby_{col}_{ascending}"
            else:
                new_col = f"ALL_UpdateTimeWeighted_{group_value}_grpby_{col}_{ascending}"
            input_df['lag'] = input_df.sort_values('first_created_unixtime', ascending=ascending).groupby(col)[[group_value]].shift(1)
            cum1 = input_df[[col] + ['lag', 'first_created_unixtime']].sort_values('first_created_unixtime').groupby(col).lag.agg(['cumsum'])
            cum2 = input_df[[col] + ['from_update', 'first_created_unixtime']].sort_values('first_created_unixtime').groupby(col).from_update.agg(['cumsum'])
            new_df = pd.DataFrame(cum1['cumsum'] / cum2['cumsum'])
            new_df.columns = [new_col]
            output_df = pd.concat([output_df, new_df], axis=1)

    return output_df

def aggregation_LastTimeWeighted(input_df, group_keys, group_value, ascendings):
    output_df = input_df[['id']]
    for col in group_keys:
        for ascending in ascendings:
            if input_df.cites_known.min() == 1:
                new_col = f"LastTimeWeighted_{group_value}_grpby_{col}_{ascending}"
            else:
                new_col = f"ALL_LastTimeWeighted_{group_value}_grpby_{col}_{ascending}"
            input_df['lag'] = input_df.sort_values('last_created_unixtime', ascending=ascending).groupby(col)[[group_value]].shift(1)
            cum1 = input_df[[col] + ['lag', 'last_created_unixtime']].sort_values('last_created_unixtime').groupby(col).lag.agg(['cumsum'])
            cum2 = input_df[[col] + ['from_last_created', 'last_created_unixtime']].sort_values('last_created_unixtime').groupby(col).from_last_created.agg(['cumsum'])
            new_df = pd.DataFrame(cum1['cumsum'] / cum2['cumsum'])
            new_df.columns = [new_col]
            output_df = pd.concat([output_df, new_df], axis=1)

    return output_df

def aggregation_YearDiff(input_df, group_keys):
    output_df = input_df[['id']]
    for col in group_keys:
        if input_df.cites_known.min() == 1:
            new_col = f"YearDiff_grpby_{col}"
        else:
            new_col = f"ALL_YearDiff_grpby_{col}"

        new_df = input_df[[col] + ['first_created_unixtime']].groupby(col).first_created_unixtime.transform('max') - input_df[[col] + ['first_created_unixtime']].groupby(col).first_created_unixtime.transform('min')
        new_df.columns = [new_col]
        output_df = pd.concat([output_df, new_df], axis=1)

    return output_df

In [ ]:
def get_agg_doi_cites_features(input_df):
    group_keys = GROUPBY_COLS + ['authors', 'journal-ref', 'report-no']
    group_value = 'doi_cites'
    agg_methods = ['max', 'mean', 'std']
    output_df = aggregation(input_df, 
                            group_keys=group_keys,
                            group_value=group_value,
                            agg_methods=agg_methods)
    return output_df

def get_agg_id_features(input_df):
    group_keys = GROUPBY_COLS + ['authors', 'journal-ref', 'report-no', 'doi_cites']
    group_value = 'id'
    agg_methods = ['count']
    output_df = aggregation(input_df, 
                            group_keys=group_keys,
                            group_value=group_value,
                            agg_methods=agg_methods)
    return output_df

def get_agg_CumFeat_features(input_df):
    group_keys = GROUPBY_COLS + ['authors', 'journal-ref', 'report-no']
    group_value = 'doi_cites'
    ascendings = [True, False]
    output_df = aggregation_CumFeat(input_df, 
                            group_keys=group_keys,
                            group_value=group_value,
                            ascendings=ascendings)
    return output_df

def get_agg_Multiple_CumFeat_features(input_df):
    group_keys = GROUPBY_COLS
    group_value = 'doi_cites'
    ascendings = [True, False]
    output_df = aggregation_Multiple_CumFeat(input_df, 
                            group_keys=group_keys,
                            group_value=group_value,
                            ascendings=ascendings)
    return output_df

def get_agg_FirstTimeWeighted_features(input_df):
    group_keys = GROUPBY_COLS + ['authors', 'journal-ref', 'report-no']
    group_value = 'doi_cites'
    ascendings = [True, False]
    output_df = aggregation_FirstTimeWeighted(input_df, 
                            group_keys=group_keys,
                            group_value=group_value,
                            ascendings=ascendings)
    return output_df

def get_agg_UpdateTimeWeighted_features(input_df):
    group_keys = GROUPBY_COLS + ['authors', 'journal-ref', 'report-no']
    group_value = 'doi_cites'
    ascendings = [True, False]
    output_df = aggregation_UpdateTimeWeighted(input_df, 
                            group_keys=group_keys,
                            group_value=group_value,
                            ascendings=ascendings)
    return output_df

def get_agg_LastTimeWeighted_features(input_df):
    group_keys = GROUPBY_COLS + ['authors', 'journal-ref', 'report-no']
    group_value = 'doi_cites'
    ascendings = [True, False]
    output_df = aggregation_LastTimeWeighted(input_df, 
                            group_keys=group_keys,
                            group_value=group_value,
                            ascendings=ascendings)
    return output_df

def get_agg_YearDiff_features(input_df):
    group_keys = GROUPBY_COLS + ['authors', 'journal-ref', 'report-no']
    output_df = aggregation_YearDiff(input_df, 
                            group_keys=group_keys)
    return output_df

In [ ]:
def groupby_preprocess(input_df):

    process_blocks = [
        get_agg_doi_cites_features,
        get_agg_id_features,
        get_agg_CumFeat_features,
        get_agg_Multiple_CumFeat_features,
        get_agg_FirstTimeWeighted_features,
        get_agg_UpdateTimeWeighted_features,
        get_agg_LastTimeWeighted_features,
        get_agg_YearDiff_features,
    ]

    output_df = input_df[['id']]
    for func in tqdm(process_blocks):
        _df = func(input_df)
        output_df = pd.merge(output_df, _df, how='left', on='id')

    return output_df

In [ ]:
with timer("ALL_df groupby features"):
    ALL_df_groupby_feat = groupby_preprocess(ALL_df)

  0%|          | 0/8 [00:00<?, ?it/s]

[ALL_df groupby features] start


100%|██████████| 8/8 [09:47<00:00, 73.43s/it]

[ALL_df groupby features] done in 587 s


In [ ]:
groupby_feat = pd.merge(df[['id']], ALL_df_groupby_feat, on='id', how='left')

In [ ]:
with timer("df groupby features"):
    df_groupby_feat = groupby_preprocess(df)

  0%|          | 0/8 [00:00<?, ?it/s]

[df groupby features] start


100%|██████████| 8/8 [00:32<00:00,  4.12s/it]

[df groupby features] done in 33 s


In [ ]:
groupby_feat = pd.merge(groupby_feat, df_groupby_feat, on='id', how='left')

In [ ]:
groupby_feat.head(3)

,id,ALL_max_doi_cites_grpby_submitter,ALL_max_doi_cites_grpby_categories,ALL_max_doi_cites_grpby_license,ALL_max_doi_cites_grpby_id_feat,ALL_max_doi_cites_grpby_doi_feat,ALL_max_doi_cites_grpby_doi_feat_2,ALL_max_doi_cites_grpby_authors,ALL_max_doi_cites_grpby_journal-ref,ALL_max_doi_cites_grpby_report-no,ALL_mean_doi_cites_grpby_submitter,ALL_mean_doi_cites_grpby_categories,ALL_mean_doi_cites_grpby_license,ALL_mean_doi_cites_grpby_id_feat,ALL_mean_doi_cites_grpby_doi_feat,ALL_mean_doi_cites_grpby_doi_feat_2,ALL_mean_doi_cites_grpby_authors,ALL_mean_doi_cites_grpby_journal-ref,ALL_mean_doi_cites_grpby_report-no,ALL_std_doi_cites_grpby_submitter,ALL_std_doi_cites_grpby_categories,ALL_std_doi_cites_grpby_license,ALL_std_doi_cites_grpby_id_feat,ALL_std_doi_cites_grpby_doi_feat,ALL_std_doi_cites_grpby_doi_feat_2,ALL_std_doi_cites_grpby_authors,ALL_std_doi_cites_grpby_journal-ref,ALL_std_doi_cites_grpby_report-no,ALL_count_id_grpby_submitter,ALL_count_id_grpby_categories,ALL_count_id_grpby_license,ALL_count_id_grpby_id_feat,ALL_count_id_grpby_doi_feat,ALL_count_id_grpby_doi_feat_2,ALL_count_id_grpby_authors,ALL_count_id_grpby_journal-ref,ALL_count_id_grpby_report-no,ALL_count_id_grpby_doi_cites,ALL_CumFeat_doi_cites_grpby_submitter_True,ALL_CumFeat_doi_cites_grpby_submitter_False,...,UpdateTimeWeighted_doi_cites_grpby_license_False,UpdateTimeWeighted_doi_cites_grpby_id_feat_True,UpdateTimeWeighted_doi_cites_grpby_id_feat_False,UpdateTimeWeighted_doi_cites_grpby_doi_feat_True,UpdateTimeWeighted_doi_cites_grpby_doi_feat_False,UpdateTimeWeighted_doi_cites_grpby_doi_feat_2_True,UpdateTimeWeighted_doi_cites_grpby_doi_feat_2_False,UpdateTimeWeighted_doi_cites_grpby_authors_True,UpdateTimeWeighted_doi_cites_grpby_authors_False,UpdateTimeWeighted_doi_cites_grpby_journal-ref_True,UpdateTimeWeighted_doi_cites_grpby_journal-ref_False,UpdateTimeWeighted_doi_cites_grpby_report-no_True,UpdateTimeWeighted_doi_cites_grpby_report-no_False,LastTimeWeighted_doi_cites_grpby_submitter_True,LastTimeWeighted_doi_cites_grpby_submitter_False,LastTimeWeighted_doi_cites_grpby_categories_True,LastTimeWeighted_doi_cites_grpby_categories_False,LastTimeWeighted_doi_cites_grpby_license_True,LastTimeWeighted_doi_cites_grpby_license_False,LastTimeWeighted_doi_cites_grpby_id_feat_True,LastTimeWeighted_doi_cites_grpby_id_feat_False,LastTimeWeighted_doi_cites_grpby_doi_feat_True,LastTimeWeighted_doi_cites_grpby_doi_feat_False,LastTimeWeighted_doi_cites_grpby_doi_feat_2_True,LastTimeWeighted_doi_cites_grpby_doi_feat_2_False,LastTimeWeighted_doi_cites_grpby_authors_True,LastTimeWeighted_doi_cites_grpby_authors_False,LastTimeWeighted_doi_cites_grpby_journal-ref_True,LastTimeWeighted_doi_cites_grpby_journal-ref_False,LastTimeWeighted_doi_cites_grpby_report-no_True,LastTimeWeighted_doi_cites_grpby_report-no_False,first_created_unixtime_y,first_created_unixtime_y,first_created_unixtime_y,first_created_unixtime_y,first_created_unixtime_y,first_created_unixtime_y,first_created_unixtime_y,first_created_unixtime_y,first_created_unixtime_y
0,1403.7138,59.0,2969,8993.0,1235,3226.0,NaN,8,NaN,NaN,13.348837,18.713850,17.493107,21.941743,17.050760,NaN,8.0,NaN,NaN,12.328603,39.091491,56.426351,53.437303,40.378667,NaN,NaN,NaN,NaN,43.0,18375,599833.0,4566,29728.0,NaN,1,NaN,NaN,26409,18.916667,18.666667,...,1.233097e-07,1.236135e-07,1.240167e-07,1.733075e-07,1.732415e-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.865038e-08,2.208467e-08,1.240262e-07,1.241811e-07,9.736840e-08,9.737710e-08,9.014459e-08,9.046495e-08,9.769856e-08,9.765968e-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,396146927.0,365972352.0,857412736.0,11891792.0,862855456.0,NaN,0.0,NaN,NaN
1,1405.5857,122.0,973,8993.0,2275,6285.0,NaN,122,122.0,NaN,46.437500,25.928335,17.493107,19.699451,24.013587,NaN,122.0,122.0,NaN,36.720510,51.695837,56.426351,51.059426,68.625551,NaN,NaN,NaN,NaN,16.0,907,599833.0,4372,105982.0,NaN,1,1.0,NaN,350,41.400000,NaN,...,1.235090e-07,1.650967e-07,1.676729e-07,1.184736e-07,1.185550e-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [ ]:
groupby_feat.to_csv(OUTDIR / 'groupby_feat.csv', index=False)